In [42]:
import rdflib
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS

# create an empty Graph
g = rdflib.ConjunctiveGraph()

# parse a local RDF file by specifying the format
result = g.parse("Desktop/dhdk_epds/resources/artchives.nq", format='nquads')

# bind the uncommon namespaces
wd = Namespace("http://www.wikidata.org/entity/") # remember that a prefix matches a URI until the last slash (or hashtag #)
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")
rdfs = Namespace ("http://www.w3.org/2000/01/")

# Get the list of art historians in our graph "g"
arthistorians_list = set()

# iterate over the triples in the graph
for s,p,o in g.triples(( None, wdt.P170, None)):   # people "o" are the creator "wdt.P170" of a collection "s"
    if "wikidata.org/entity/" in str(o):           # look for the substring to filter wikidata entities only
        arthistorians_list.add('<' + str(o) + '>')     # remember to transform them in strings! 
    
print(arthistorians_list)


{'<http://www.wikidata.org/entity/Q457739>', '<http://www.wikidata.org/entity/Q3051533>', '<http://www.wikidata.org/entity/Q6700132>', '<http://www.wikidata.org/entity/Q2824734>', '<http://www.wikidata.org/entity/Q537874>', '<http://www.wikidata.org/entity/Q3057287>', '<http://www.wikidata.org/entity/Q85761254>', '<http://www.wikidata.org/entity/Q41616785>', '<http://www.wikidata.org/entity/Q1712683>', '<http://www.wikidata.org/entity/Q19997512>', '<http://www.wikidata.org/entity/Q1641821>', '<http://www.wikidata.org/entity/Q1296486>', '<http://www.wikidata.org/entity/Q61913691>', '<http://www.wikidata.org/entity/Q90407>', '<http://www.wikidata.org/entity/Q60185>', '<http://www.wikidata.org/entity/Q1373290>', '<http://www.wikidata.org/entity/Q1271052>', '<http://www.wikidata.org/entity/Q1715096>', '<http://www.wikidata.org/entity/Q55453618>', '<http://www.wikidata.org/entity/Q995470>', '<http://www.wikidata.org/entity/Q18935222>', '<http://www.wikidata.org/entity/Q1629748>', '<http://w

In [44]:
from SPARQLWrapper import SPARQLWrapper, JSON
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# get the endpoint API
wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"

# prepare the values to be queried
historians = ' '.join(arthistorians_list) # <uri1> <uri2> <uri3> ... <uriN>

# prepare the query: for each historian in ARTchives check in wikidata if there are work or education places.
formationplace_query = """ 
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
SELECT DISTINCT ?historian ?workplace ?workplace_label ?coordinates1 ?eduplace ?eduplace_label ?coordinates2 
WHERE {
    VALUES ?historian {"""+historians+"""} . 
    optional {?historian wdt:P108 ?workplace . ?workplace rdfs:label ?workplace_label .
    FILTER (langMatches(lang(?workplace_label), "EN")) optional {?workplace wdt:P625 ?coordinates1}}
    optional {?historian wdt:P69 ?eduplace . 
    ?eduplace rdfs:label ?eduplace_label .
    FILTER (langMatches(lang(?eduplace_label), "EN")) optional {?eduplace wdt:P625 ?coordinates2 }}
    } 
    
"""

# set the endpoint 
sparql_wd = SPARQLWrapper(wikidata_endpoint)
# set the query
sparql_wd.setQuery(formationplace_query)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results = sparql_wd.query().convert()
duplicate = []
# manipulate the result
for result in results["results"]["bindings"]:
    print(result)
    historian_uri = result["historian"]["value"]
    #print("historian:", historian_uri)
    if "workplace" in result: 
        birthplace = result["workplace"]["value"]
        if "workplace_label" in result: 
            birthplace_label = result["workplace_label"]["value"]
            #print("work:", birthplace, birthplace_label)

                    # only if both uri and label are found we add them to 
            g.add(( URIRef(historian_uri) , URIRef(wdt.P108) , URIRef(birthplace) ))
            g.add(( URIRef(birthplace) , RDFS.label , Literal(birthplace_label) ))
            if "coordinates1" in result:
                work_coord = result["coordinates1"]["value"][6:-1].split(" ")
                g.add(( URIRef(birthplace) , URIRef(wdt.P625) , Literal(work_coord) ))
    if "eduplace" in result: 
        eduplace = result["eduplace"]["value"]
        if "eduplace_label" in result: 
            eduplace_label = result["eduplace_label"]["value"]
            print("education:", eduplace, eduplace_label)
                    # only if both uri and label are found we add them to the graph
            g.add(( URIRef(historian_uri) , URIRef(wdt.P69) , URIRef(eduplace) ))
            g.add(( URIRef(eduplace) , RDFS.label , Literal(eduplace_label) ))
            if "coordinates2" in result:
                eduplace_coord = result["coordinates2"]["value"][6:-1].split(" ")
                g.add(( URIRef(eduplace) , URIRef(wdt.P625) , Literal(eduplace_coord) ))


{'historian': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q19997512'}, 'eduplace': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q13371'}, 'eduplace_label': {'xml:lang': 'en', 'type': 'literal', 'value': 'Harvard University'}, 'coordinates2': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral', 'type': 'literal', 'value': 'Point(-71.116943888 42.374443888)'}}
education: http://www.wikidata.org/entity/Q13371 Harvard University
{'historian': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q19997512'}, 'eduplace': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q13371'}, 'eduplace_label': {'xml:lang': 'en-ca', 'type': 'literal', 'value': 'Harvard University'}, 'coordinates2': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral', 'type': 'literal', 'value': 'Point(-71.116943888 42.374443888)'}}
education: http://www.wikidata.org/entity/Q13371 Harvard University
{'historian': {'type': 'uri', 'value': 'http://www.wikidata.org/ent

{'historian': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q88907'}, 'eduplace': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q55044'}, 'eduplace_label': {'xml:lang': 'en-gb', 'type': 'literal', 'value': 'Ludwig Maximilian University of Munich'}, 'coordinates2': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral', 'type': 'literal', 'value': 'Point(11.580277777 48.150833333)'}, 'workplace': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q13371'}, 'workplace_label': {'xml:lang': 'en', 'type': 'literal', 'value': 'Harvard University'}, 'coordinates1': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral', 'type': 'literal', 'value': 'Point(-71.116943888 42.374443888)'}}
education: http://www.wikidata.org/entity/Q55044 Ludwig Maximilian University of Munich
{'historian': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q88907'}, 'eduplace': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q55044'}, 'eduplace_label': {'x

education: http://www.wikidata.org/entity/Q213439 University of Virginia
{'historian': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q19997512'}, 'eduplace': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q213439'}, 'eduplace_label': {'xml:lang': 'en-ca', 'type': 'literal', 'value': 'University of Virginia'}, 'coordinates2': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral', 'type': 'literal', 'value': 'Point(-78.503333333 38.035555555)'}}
education: http://www.wikidata.org/entity/Q213439 University of Virginia
{'historian': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q19997512'}, 'eduplace': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q213439'}, 'eduplace_label': {'xml:lang': 'en-gb', 'type': 'literal', 'value': 'University of Virginia'}, 'coordinates2': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral', 'type': 'literal', 'value': 'Point(-78.503333333 38.035555555)'}}
education: http://www.wikidata.org/entity/Q2

education: http://www.wikidata.org/entity/Q5303281 Downside School
{'historian': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q537874'}, 'eduplace': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q5303281'}, 'eduplace_label': {'xml:lang': 'en-gb', 'type': 'literal', 'value': 'Downside School'}, 'coordinates2': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral', 'type': 'literal', 'value': 'Point(-2.495 51.255)'}, 'workplace': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q35794'}, 'workplace_label': {'xml:lang': 'en-ca', 'type': 'literal', 'value': 'University of Cambridge'}, 'coordinates1': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral', 'type': 'literal', 'value': 'Point(0.117222222 52.205277777)'}}
education: http://www.wikidata.org/entity/Q5303281 Downside School
{'historian': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q457739'}, 'eduplace': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3339070'}

education: http://www.wikidata.org/entity/Q1902016 Marlborough College
{'historian': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3051533'}, 'eduplace': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1902016'}, 'eduplace_label': {'xml:lang': 'en-ca', 'type': 'literal', 'value': 'Marlborough College'}, 'coordinates2': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral', 'type': 'literal', 'value': 'Point(-1.737 51.416)'}, 'workplace': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q223429'}, 'workplace_label': {'xml:lang': 'en', 'type': 'literal', 'value': 'University of Birmingham'}, 'coordinates1': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral', 'type': 'literal', 'value': 'Point(-1.930555555 52.450555555)'}}
education: http://www.wikidata.org/entity/Q1902016 Marlborough College
{'historian': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3051533'}, 'eduplace': {'type': 'uri', 'value': 'http://www.wikidata.org/ent

In [53]:
firstdict = {}
final = {}
for hist, prop, place in g.triples((None, wdt.P69, None)):
    for eduplace, hasname, eduplacename in g.triples((place, RDFS.label, None)):
        for edup, hascoordinates, coord in g.triples((place, wdt.P625, None)):
            for historian, p, name in g.triples(( hist, RDFS.label, None)):   
                if "wikidata.org/entity/" in str(s):  
                    key = (eduplacename.strip(), "<" + place.strip() +">", float(coord.split(" ")[1][1:-2]), float(coord.split(" ")[0][2:-2]))
                    value = tuple(["<" + hist.strip() +">", name.strip()])
                    if key not in firstdict.keys():
                        firstdict[key] = set([value])
                    else:
                        firstdict[key].update([value])


                    
for hist, prop, place in g.triples((None, wdt.P108, None)):
    for workplace, hasname, workplacename in g.triples((place, RDFS.label, None)):
        for wplace, hasCoordinates, coord in g.triples((place, wdt.P625, None)):
            for historian, p, name in g.triples(( hist, RDFS.label, None)):   
                if "wikidata.org/entity/" in str(s):  
                    key = (workplacename.strip(), "<" + workplace.strip() +">", float(coord.split(" ")[1][1:-2]), float(coord.split(" ")[0][2:-2]))
                    #print(key)
                    value = tuple(["<" + hist.strip() +">", name.strip()])
                    if key not in firstdict.keys():
                        firstdict[key] = set([value])
                    else:
                        firstdict[key].update([value])


for k, v in firstdict.items(): #creation of a dictionary that has as key a tuple with the info for a place and as values a list of tuples of art historians connected to that place.
    for el in v:
        if k not in final.keys():
            final[k] = [el]
        else:
            final[k].append(el)




In [54]:
import csv

# prepare the csv file
with open('formationplaces.csv', mode='w') as my_file:
    my_writer = csv.writer(my_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    # write the column names
    my_writer.writerow(['place', 'place_uri','lat', 'long'])
    
    # access the list of tuples of the query results
    for res in prova:
        # write in the csv
        my_writer.writerow([res[0], res[1], res[2], res[3]])

In [55]:
import pandas as pd

# parse the csv into a dataframe
df = pd.read_csv("formationplaces.csv")
# print the first 5 rows
df.head()

,place,place_uri,lat,long
0,United States of America,<http://www.wikidata.org/entity/Q30>,39.828175,-98.579500
1,United States,<http://www.wikidata.org/entity/Q30>,39.828175,-98.579500
2,Hamburg,<http://www.wikidata.org/entity/Q1055>,53.550000,10.000000
3,Florence,<http://www.wikidata.org/entity/Q2044>,43.771389,11.254167
4,University of Bonn,<http://www.wikidata.org/entity/Q152171>,50.733889,7.102222


In [56]:
g.serialize(destination='Desktop/dhdk_epds/resources/artchives2.nq', format='nquads')
result = g.parse("Desktop/dhdk_epds/resources/artchives2.nq", format='nquads')



In [57]:
related = {} #dictionary to collect the main subjects of an art historian -p921- present in ARTchives data 
for s,p,o in g.triples(( None, wdt.P170, None)):   
    if "wikidata.org/entity/" in str(o):           
        for hist, prop, obj in g.triples((o, wdt.P921, None)):     
            for subj, pr, name in g.triples(( hist, RDFS.label, None)):   
                key = tuple(['<' + str(hist) + '>', name.strip()])
                value = '<' + str(obj) + '>'
                if key not in related.keys(): #the dict has as keys the historians and as values the related list of main subjects.
                    related[key] = [value]
                else:
                    related[key].append(value)
    
#for k, v in related.items():
    #print(k, v)



In [61]:
related_dict= {}
prova = {}
for k, v in related.items():  #for each art historian I extract those main subjects which are defined as places in wikidatata 
    relatedlist = ' '.join(v)


    query_results = """
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        SELECT DISTINCT ?place ?place_label ?coord
        WHERE {
            VALUES ?place {"""+relatedlist+"""} . 
            ?place rdfs:label ?place_label . 
            FILTER (langMatches(lang(?place_label), "EN")) . ?place wdt:P625 ?coord
            } 

        """

    # set the endpoint 
    sparql_wd = SPARQLWrapper(wikidata_endpoint)
    # set the query
    sparql_wd.setQuery(query_results)
    # set the returned format
    sparql_wd.setReturnFormat(JSON)
    # get the results
    results = sparql_wd.query().convert()

    for result in results["results"]["bindings"]:
        #print(result)
        place = '<' + str(result["place"]["value"]) + '>'
        place_name = str(result["place_label"]["value"]) 
        coord = result["coord"]["value"][6:-1].split(" ")
        key = tuple([place_name, place, float(coord[1]), float(coord[0])])
        for value in v:
            if value == str(key[1]):
                if key not in prova.keys():
                    prova[key] = set([k])
                else: 
                    prova[key].update([k])
                

                
for k, v in prova.items():
    for el in v:
        if k not in related_dict.keys():
            related_dict[k] = [el]
        else:
            related_dict[k].append(el)


for k, v in related_dict.items():       
    print(k, v)


('United States of America', '<http://www.wikidata.org/entity/Q30>', 39.828175, -98.5795) [('<http://www.wikidata.org/entity/Q60185>', 'Aby Warburg'), ('<http://www.wikidata.org/entity/Q1089074>', 'Federico Zeri'), ('<http://www.wikidata.org/entity/Q457739>', 'Leo Steinberg'), ('<http://www.wikidata.org/entity/Q90407>', 'Richard Krautheimer')]
('United States', '<http://www.wikidata.org/entity/Q30>', 39.828175, -98.5795) [('<http://www.wikidata.org/entity/Q60185>', 'Aby Warburg'), ('<http://www.wikidata.org/entity/Q1089074>', 'Federico Zeri'), ('<http://www.wikidata.org/entity/Q457739>', 'Leo Steinberg'), ('<http://www.wikidata.org/entity/Q90407>', 'Richard Krautheimer')]
('Hamburg', '<http://www.wikidata.org/entity/Q1055>', 53.55, 10.0) [('<http://www.wikidata.org/entity/Q60185>', 'Aby Warburg')]
('Florence', '<http://www.wikidata.org/entity/Q2044>', 43.771388888, 11.254166666) [('<http://www.wikidata.org/entity/Q60185>', 'Aby Warburg'), ('<http://www.wikidata.org/entity/Q55453618>', 

In [63]:
from ipywidgets import HTML

from ipyleaflet import Map, Marker, Popup, LayersControl

center = (41.080684, -30.683374)


m = Map(center=center, zoom=3, close_popup_on_click=False)



for k,v in related_dict1.items():
    marker = Marker(location=(k[2], k[3]))
    m.add_layer(marker)
    message = HTML()
    marker.popup = message
    message.description = ""
    namelist = str()
    for value in v:
        namelist= namelist + "<a href='http://artchives.fondazionezeri.unibo.it/historian-" + value[0][32:-1] + "'>" + value[1] + "</a>" + ". "
    message.value = "<b>" + k[0] + "</b>" + "<br>"  + namelist





m

Map(center=[41.080684, -30.683374], close_popup_on_click=False, controls=(ZoomControl(options=['position', 'zo…

In [69]:
hist_dict = {} #dict that has as keys art historians and as values the list of related places. 
hist_list = []
for k, v in related_dict.items():
    for value in v:
        if value not in hist_list:
            hist_list.append(value)
    for el in hist_list:
        if el in v:
            if el not in hist_dict.keys():
                hist_dict[el] = [k]
            else:
                hist_dict[el].append(k)
        

for k, v in hist_dict.items():            
    print(k, v)

('<http://www.wikidata.org/entity/Q60185>', 'Aby Warburg') [('United States of America', '<http://www.wikidata.org/entity/Q30>', 39.828175, -98.5795), ('United States', '<http://www.wikidata.org/entity/Q30>', 39.828175, -98.5795), ('Hamburg', '<http://www.wikidata.org/entity/Q1055>', 53.55, 10.0), ('Florence', '<http://www.wikidata.org/entity/Q2044>', 43.771388888, 11.254166666), ('University of Bonn', '<http://www.wikidata.org/entity/Q152171>', 50.733888888, 7.102222222), ('Humboldt University of Berlin', '<http://www.wikidata.org/entity/Q152087>', 52.518055555, 13.393333333), ('Ludwig Maximilian University of Munich', '<http://www.wikidata.org/entity/Q55044>', 48.150833333, 11.580277777), ('University of Strasbourg', '<http://www.wikidata.org/entity/Q157575>', 48.580277777, 7.764444444)]
('<http://www.wikidata.org/entity/Q1089074>', 'Federico Zeri') [('United States of America', '<http://www.wikidata.org/entity/Q30>', 39.828175, -98.5795), ('United States', '<http://www.wikidata.org/

In [21]:
from ipyleaflet import LayerGroup
center = (41.080684, -30.683374)


m = Map(center=center, zoom=3, close_popup_on_click=False)
control = LayersControl(position='topright')
m.add_control(control)

for k,v in hist_dict.items():
    layer_group = LayerGroup(layers=(), name=k[1]) #creating different layers for different historians
    m.add_layer(layer_group)
    for value in v:
        marker = Marker(location=(value[2], value[3]))
        message2 = HTML()
        marker.popup = message2
        message2.description = ""
        message2.value ="<b>" + "<a href='http://artchives.fondazionezeri.unibo.it/historian-" + k[0][32:-1] +"'>" + k[1] + "</a>" + "</b>" + "<br>"  + value[0]
        layer_group.add_layer(marker)





m

Map(center=[41.080684, -30.683374], close_popup_on_click=False, controls=(ZoomControl(options=['position', 'zo…

In [64]:
people = []
for k, v in related.items():   
    relatedlist = ' '.join(v)
    #print(relatedlist)
    query_res = """
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX wd: <http://www.wikidata.org/entity/>
        SELECT DISTINCT ?person ?person_label ?occupation
        WHERE {
            VALUES ?person {"""+relatedlist+"""} . 
            ?person wdt:P31 wd:Q5; rdfs:label ?person_label . 
            VALUES ?occupation {wd:Q1792450 wd:Q201788 wd:Q36180 wd:Q4164507 wd:Q1126160 wd:Q10732476} . 
            ?person wdt:P106 ?occupation .
            FILTER (langMatches(lang(?person_label), "EN") ) 
            } 

        """

    # set the endpoint 
    sparql_wd = SPARQLWrapper(wikidata_endpoint)
    # set the query
    sparql_wd.setQuery(query_res)
    # set the returned format
    sparql_wd.setReturnFormat(JSON)
    # get the results
    results = sparql_wd.query().convert()

    for result in results["results"]["bindings"]:
        #print(result)
        person = '<' + str(result["person"]["value"]) + '>'
        person_name = str(result["person_label"]["value"])
        key = tuple([person_name, person])
        for value in v:
            if value == str(key[1]):
                tupla = tuple([k[1], k[0], person_name, person, 1])
                if tupla not in people:
                    people.append(tupla)

            
print(people)

[('Aby Warburg', '<http://www.wikidata.org/entity/Q60185>', 'Jacob Burckhardt', '<http://www.wikidata.org/entity/Q59945>', 1), ('Aby Warburg', '<http://www.wikidata.org/entity/Q60185>', 'Hermann Usener', '<http://www.wikidata.org/entity/Q63078>', 1), ('Aby Warburg', '<http://www.wikidata.org/entity/Q60185>', 'Vitruvius', '<http://www.wikidata.org/entity/Q47163>', 1), ('Aby Warburg', '<http://www.wikidata.org/entity/Q60185>', 'Erwin Panofsky', '<http://www.wikidata.org/entity/Q57656>', 1), ('Aby Warburg', '<http://www.wikidata.org/entity/Q60185>', 'Bernard Berenson', '<http://www.wikidata.org/entity/Q359047>', 1), ('Aby Warburg', '<http://www.wikidata.org/entity/Q60185>', 'Herbert Horne', '<http://www.wikidata.org/entity/Q597283>', 1), ('Aby Warburg', '<http://www.wikidata.org/entity/Q60185>', 'Isolde Kurz', '<http://www.wikidata.org/entity/Q77856>', 1), ('Aby Warburg', '<http://www.wikidata.org/entity/Q60185>', 'Carl Justi', '<http://www.wikidata.org/entity/Q77314>', 1), ('Aby Warburg'

In [17]:
coll_related = {}
for coll,p,hist in g.triples(( None, wdt.P170, None)):
    for historian, proper, hist_name in g.triples((hist, RDFS.label, None)):     # people "o" are the creator "wdt.P170" of a collection "s"
        for collection, pr, coll_name in g.triples(( coll, RDFS.label, None)):   
            for col, prop, content in g.triples((coll, wdt.P921, None)):     
                for cont, pro, content_label in g.triples(( content, RDFS.label, None)): 
                    if "wikidata.org/entity/" in str(content):    
                        key = tuple([hist_name.strip(), '<' + str(hist) + '>'])
                        value = '<' + str(content) + '>'
                        if key not in coll_related.keys():
                            coll_related[key] = set([value])
                        else:
                            coll_related[key].add(value)
    


  
for k, v in coll_related.items():
    print( k, v)
    

#print(collrelatedlist)
 


('Julius S. Held', '<http://www.wikidata.org/entity/Q1712683>') {'<http://www.wikidata.org/entity/Q7016>', '<http://www.wikidata.org/entity/Q6262872>', '<http://www.wikidata.org/entity/Q1425008>', '<http://www.wikidata.org/entity/Q457739>', '<http://www.wikidata.org/entity/Q533444>', '<http://www.wikidata.org/entity/Q5598>', '<http://www.wikidata.org/entity/Q4351610>', '<http://www.wikidata.org/entity/Q57656>', '<http://www.wikidata.org/entity/Q270658>', '<http://www.wikidata.org/entity/Q59533353>', '<http://www.wikidata.org/entity/Q214493>', '<http://www.wikidata.org/entity/Q11203476>', '<http://www.wikidata.org/entity/Q3305213>', '<http://www.wikidata.org/entity/Q6831576>', '<http://www.wikidata.org/entity/Q150679>', '<http://www.wikidata.org/entity/Q2185042>', '<http://www.wikidata.org/entity/Q33506>', '<http://www.wikidata.org/entity/Q100511>', '<http://www.wikidata.org/entity/Q3137182>', '<http://www.wikidata.org/entity/Q555132>', '<http://www.wikidata.org/entity/Q19997518>', '<ht

In [65]:
for k, v in coll_related.items(): 
    coll_list = ' '.join(v)
    query_res = """
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX wd: <http://www.wikidata.org/entity/>
        SELECT DISTINCT ?person ?person_label ?occupation
        WHERE {
            VALUES ?person {"""+coll_list+"""} . 
            ?person wdt:P31 wd:Q5; rdfs:label ?person_label .
            VALUES ?occupation {wd:Q1792450 wd:Q201788 wd:Q36180 wd:Q4164507 wd:Q1126160 wd:Q10732476} . 
            ?person wdt:P106 ?occupation .
            FILTER (langMatches(lang(?person_label), "EN")) . 
            } 

        """

    # set the endpoint 
    sparql_wd = SPARQLWrapper(wikidata_endpoint)
    # set the query
    sparql_wd.setQuery(query_res)
    # set the returned format
    sparql_wd.setReturnFormat(JSON)
    # get the results
    results = sparql_wd.query().convert()

    for result in results["results"]["bindings"]:
        person = '<' + str(result["person"]["value"]) + '>'
        person_name = str(result["person_label"]["value"]) 
        key = tuple([person_name, person])
        #print(death)
        for value in v:
            if value == str(key[1]):
                tupla = tuple([k[0], k[1], person_name, person, 2])
                if tupla not in people:
                    people.append(tupla)

for el in people:           
    print(el)

('Aby Warburg', '<http://www.wikidata.org/entity/Q60185>', 'Jacob Burckhardt', '<http://www.wikidata.org/entity/Q59945>', 1)
('Aby Warburg', '<http://www.wikidata.org/entity/Q60185>', 'Hermann Usener', '<http://www.wikidata.org/entity/Q63078>', 1)
('Aby Warburg', '<http://www.wikidata.org/entity/Q60185>', 'Vitruvius', '<http://www.wikidata.org/entity/Q47163>', 1)
('Aby Warburg', '<http://www.wikidata.org/entity/Q60185>', 'Erwin Panofsky', '<http://www.wikidata.org/entity/Q57656>', 1)
('Aby Warburg', '<http://www.wikidata.org/entity/Q60185>', 'Bernard Berenson', '<http://www.wikidata.org/entity/Q359047>', 1)
('Aby Warburg', '<http://www.wikidata.org/entity/Q60185>', 'Herbert Horne', '<http://www.wikidata.org/entity/Q597283>', 1)
('Aby Warburg', '<http://www.wikidata.org/entity/Q60185>', 'Isolde Kurz', '<http://www.wikidata.org/entity/Q77856>', 1)
('Aby Warburg', '<http://www.wikidata.org/entity/Q60185>', 'Carl Justi', '<http://www.wikidata.org/entity/Q77314>', 1)
('Aby Warburg', '<http:

In [66]:
with open('people.csv', mode='w') as my_file:
    my_writer = csv.writer(my_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    # write the column names
    my_writer.writerow(['art_hist', 'art_hist_uri','person', 'person_uri', 'weight'])
    
    # access the list of tuples of the query results
    for res in people:
        # write in the csv
        my_writer.writerow([res[0], res[1], res[2], res[3], res[4]])

In [67]:
# parse the csv into a dataframe
df = pd.read_csv("people.csv")
# print the first 5 rows
df.head()

,art_hist,art_hist_uri,person,person_uri,weight
0,Aby Warburg,<http://www.wikidata.org/entity/Q60185>,Jacob Burckhardt,<http://www.wikidata.org/entity/Q59945>,1
1,Aby Warburg,<http://www.wikidata.org/entity/Q60185>,Hermann Usener,<http://www.wikidata.org/entity/Q63078>,1
2,Aby Warburg,<http://www.wikidata.org/entity/Q60185>,Vitruvius,<http://www.wikidata.org/entity/Q47163>,1
3,Aby Warburg,<http://www.wikidata.org/entity/Q60185>,Erwin Panofsky,<http://www.wikidata.org/entity/Q57656>,1
4,Aby Warburg,<http://www.wikidata.org/entity/Q60185>,Bernard Berenson,<http://www.wikidata.org/entity/Q359047>,1


In [68]:
from pyvis import network as net


people_net = net.Network(height="750px", width="100%", bgcolor="white", font_color="white", notebook="True")

# set the physics layout of the network
people_net.barnes_hut()
people_data = pd.read_csv("people.csv")

sources = people_data['art_hist']
targets = people_data['person']
weights = people_data['weight']

edge_data = zip(sources, targets, weights)

for e in edge_data:
    print(e)
    print(e)
    src = e[0]
    dst = e[1]
    w = e[2]

    people_net.add_node(src, src, title=src, color= "#1cae81")
    people_net.add_node(dst, dst, title=dst, color= "#1cae81")
    if w == 1:
        people_net.add_edge(src, dst, value=w, color="#23daa2")
    elif w ==2:
        people_net.add_edge(src, dst, value=w, color="gainsboro")

neighbor_map = people_net.get_adj_list()

# add neighbor data to node hover data
for node in people_net.nodes:
    node["title"] += " relations:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]])

people_net.show("people.html")

('Aby Warburg', 'Jacob Burckhardt', 1)
('Aby Warburg', 'Jacob Burckhardt', 1)
('Aby Warburg', 'Hermann Usener', 1)
('Aby Warburg', 'Hermann Usener', 1)
('Aby Warburg', 'Vitruvius', 1)
('Aby Warburg', 'Vitruvius', 1)
('Aby Warburg', 'Erwin Panofsky', 1)
('Aby Warburg', 'Erwin Panofsky', 1)
('Aby Warburg', 'Bernard Berenson', 1)
('Aby Warburg', 'Bernard Berenson', 1)
('Aby Warburg', 'Herbert Horne', 1)
('Aby Warburg', 'Herbert Horne', 1)
('Aby Warburg', 'Isolde Kurz', 1)
('Aby Warburg', 'Isolde Kurz', 1)
('Aby Warburg', 'Carl Justi', 1)
('Aby Warburg', 'Carl Justi', 1)
('Aby Warburg', 'Hubert Janitschek', 1)
('Aby Warburg', 'Hubert Janitschek', 1)
('Aby Warburg', 'Jacques Mesnil', 1)
('Aby Warburg', 'Jacques Mesnil', 1)
('Ernst Steinmann', 'Rudolf Wittkower', 1)
('Ernst Steinmann', 'Rudolf Wittkower', 1)
('Ernst Steinmann', 'Napoleon', 1)
('Ernst Steinmann', 'Napoleon', 1)
('Ernst Steinmann', 'Anton Heinrich Springer', 1)
('Ernst Steinmann', 'Anton Heinrich Springer', 1)
('Ernst Steinman

('Richard Krautheimer', 'Ernst Steinmann', 2)
('Richard Krautheimer', 'Ernst Steinmann', 2)
('Richard Krautheimer', 'Werner Weisbach', 2)
('Richard Krautheimer', 'Werner Weisbach', 2)
('Richard Krautheimer', 'Wolfgang Lotz', 2)
('Richard Krautheimer', 'Wolfgang Lotz', 2)
('John Pope-Hennessy', 'Una Pope-Hennessy', 2)
('John Pope-Hennessy', 'Una Pope-Hennessy', 2)
('John Pope-Hennessy', 'James Pope-Hennessy', 2)
('John Pope-Hennessy', 'James Pope-Hennessy', 2)
('John Pope-Hennessy', 'John Pope-Hennessy', 2)
('John Pope-Hennessy', 'John Pope-Hennessy', 2)
('John Pope-Hennessy', 'Donatello', 2)
('John Pope-Hennessy', 'Donatello', 2)
('Werner Cohn', 'Friedrich Nietzsche', 2)
('Werner Cohn', 'Friedrich Nietzsche', 2)
('Werner Cohn', 'Thomas Mann', 2)
('Werner Cohn', 'Thomas Mann', 2)
('Werner Cohn', 'Jacob Burckhardt', 2)
('Werner Cohn', 'Jacob Burckhardt', 2)
('Werner Cohn', 'Julius von Schlosser', 2)
('Werner Cohn', 'Julius von Schlosser', 2)
('Werner Cohn', 'Lionello Venturi', 2)
('Werne